COMP 215 - LAB 6 (Game of Life)
-----------------------------------------------
#### Name: David Lu
#### Date: 1 Mar - 8 Mar

This lab exercise is an introduction to [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) discovered and studied by [John Conway](https://en.wikipedia.org/wiki/John_Horton_Conway).

**New Python concepts**:
   * 2D numpy [array](https://numpy.org/doc/stable/reference/generated/numpy.array.html)
   * basic [array operations](https://www.pluralsight.com/guides/overview-basic-numpy-operations)

**New Computation concepts**:
   * [2D sliding dot product](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate2d.html)
   * [Animation](https://matplotlib.org/stable/api/animation_api.html)

**Credits**:
    Code examples based on [Think Complexity, 2nd edition](https://thinkcomplex.com).
    Copyright 2016 Allen Downey, [MIT License](http://opensource.org/licenses/MIT)

As usual, the first code cell simply imports all the modules we'll be using...

In [2]:
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from scipy.signal import correlate2d

%matplotlib inline
# Configure matplotlib's animation library to work in the browser.
matplotlib.rc('animation', html='jshtml')

A reusable function for plotting 2D numpy.array as an image...

In [3]:
def plot_2d_array(array, axes=None, title='', cmap='Blues', **options):
    """
    Plot the 2D array as an image on the given axes  1's will be dark blue, 0's will be light blue.

    :param axes: the axes to plot on, or None to use the `plt.gca()` (current axes)
    :param options: keyword arguments passed directly to `plt.imshow()`
           see https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.imshow.html
    """
    axes = axes or plt.gca()  # If not axes are provided, draw on current axes
    axes.set_title(title)
    # Turn off axes labels and tick marks
    axes.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False ,
                     labelbottom=False, labeltop=False, labelleft=False, labelright=False,)
    # Defaults for displaying a "matrix" with hard-pixel boundaries and (0,0) at top-left
    options = {**dict(interpolation='nearest', origin='upper'), **options}
    axes.imshow(array, cmap=cmap, **options)

## Life Fundamentals
Both the textbook and our week6 notebooks develop some fundamental operations for simulating the Game of Life.
Some of the code from `week6.ipynb` is reproduced here without deriving all the ideas - review `week6.ipynb` notebook for the full story.
The sliding dot product algorithm in `step` is implemented with [scipy.signal.correlate2d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate2d.html).  It's behaviour is identical to the derivation in `week6.ipynb`.

#### Notes
* we use `correlate2d(state, kernel, mode='same', boundary='wrap')` to pad state matrix with 1 value around perimeter from the opposite side of the matrix.
* since the rule is a 1D `np.array`, we use "vector indexing" here to set the 'live' state transitions:  `next_state[[3, 12, 13]] = 1`

In [4]:
class Life2D:
    """ An basic 2D Cellular Automata that implementes Conway's Game of Life """
    kernel = np.array([[1, 1, 1],
                       [1, 10, 1],
                       [1, 1, 1]])

    next_state = np.zeros(19, dtype=np.uint8)
    next_state[[3, 12, 13]] = 1

    def __init__(self, n_rows, n_cols=None):
        """ Construct a n_rows x n_cols 2D CA """
        n_cols = n_cols or n_rows  # i.e., n_cols if n_cols is not None and n_cols != 0 else n_rows
        self.state = np.zeros(shape=(n_rows, n_cols), dtype=np.uint8)

    def step(self):
        """Executes one time step. """
        transitions = correlate2d(self.state, self.kernel, mode='same', boundary='wrap')
        self.state = self.next_state[transitions]

    def draw(self, axes=None, step=''):
        axes = axes or plt.gca()   # use pyplot's current axes if none are provided.
        plot_2d_array(self.state, axes, title=f"Conway's Game of Life {step}")

# Some comments for myself:
#
# <n_cols or n_rows>  ->  short-circuit evaluation: returns the first "truthy" value encountered,
#                         otherwise, if both values are "falsy", return the last value in the expression
# <n_cols and n_rows> ->  short-circuit evaluation: returns the first "falsy" value encountered,
#                         otherwise, if both values for "truthy", return the last value in the expression
#
# interface: np.zeros(<shape>, <dtype>)  -->  <shape> can be int (for single-dimension arrays), or tuples (for multidimensional arrays)
#                                        -->  <dtype> is optional, default is numpy.float64
#
# the correlation step encodes the neighbourhood environment of each cell as a single number in the range (0-8 and 10-18);
# this number is then referenced in the lookup table, to determine how each cell will evolve in the coming step

### Visualizing a 2D simulation model
For 1D CAs, we could use a 2D image to visualize the CA progression over time.
For 2D CAs, we could try using time as a 3rd axis to generate a 3D visualziation, but without VR, its hard to explore a 3D visualization.
Instead, we'll use a simple 2D ["stop motion" animation](https://en.wikipedia.org/wiki/Animation).
Here is a generic class that animates any 2D model that provides step() and draw() methods, and uses matplotlib to render its state.

See [Matplotlib animation in Jupyter](https://www.allendowney.com/blog/2019/07/25/matplotlib-animation-in-jupyter/) by our textbook author for a discussion of the pros and cons of various animation techniques in jupyter notebooks.

In [5]:
class Animation2D:
    """
      Animates any 2D model with a step() method and a draw() method, using matplotlib
      model.step() should take no parameters - just step the model forward one step.
      model.draw() should take 2 parameters, the matpltolib axes to draw on and an integer step number
    """

    def __init__(self, model, frames=50, figsize=(8, 8)):
        """
        :param model: the simulation object to animate, with step() and draw(axes, step) methods
        :param frames: number of animation frames to generate
        """
        self.model = model
        self.frames = frames
        self.fig, self.ax = plt.subplots(figsize=figsize)

    def animation_step(self, step):
        """ Step the model forward and draw the plot """
        if step > 0:
            self.model.step()
        self.model.draw(self.ax, step=step)

    def show(self):
        """ return the matplotlib animation object, ready for display """
        anim = animation.FuncAnimation(self.fig, self.animation_step, frames=self.frames)
        plt.close()  # this ensures the last frame is not shown as a separate plot
        return anim

    def animate(self, interval=None):
        """ Animate the model simulation directly in the notebook display block """
        from IPython.display import clear_output
        try:
            for i in range(self.frames):
                clear_output(wait=True)  # clear the IPython display
                self.ax.clear()          # clear old image from the axes (fixes a performance issue)
                plt.figure(self.fig)     # add the figure back to pyplot ** sigh **
                self.animation_step(i)
                plt.show()               # show the current animation frame (pyplot then closes and throws away figure ** sigh **)
                if interval:
                    time.sleep(interval)
        except KeyboardInterrupt:
            pass

### Sample usage
Animate a single glider...

In [ ]:
size = 10
life = Life2D(size)
glider = np.array([[0, 1, 0], [0, 0, 1], [1, 1, 1]], dtype=np.uint8)
life.state[1:4, 2:5] = glider             # bad idea to alter an objects state directly!  We'll improve it in Ex. 3
gol = Animation2D(life, frames=20)
gol.animate()

# Some explanations for myself:
#
# The <n_rows> of the 2D cellular automaton is set to 10.
# since no <n_cols> was provided, it is set to the same value as <n_rows>  -->  also 10
#
# Because the <life2D> class doesn't yet have a method to modify the <state> from its initial configuration of all zeros,
# we're forced (for now) to modify this state directly by assigning the shape of the glider to a slice/subset of the state array
#
# In line 4, we pass the <life> object to the <Animate2D> constructor as the model to animate
# Line 5 calls the <Animate2D.animate()>. An altnernative would be to call <Animate2D.show()>, which is quite cool.

## Exercise 1:  Enumerate all Elementary 2D CA's
Write a line or 2 of code to compute and print the answer for each question:
* how many unique states are there for a 3x3 (9 cell) neighbourhood?
* how many unique “rules” are there? (where a rule provides a mapping for each possible neighbourhood state to the output for the centre cell)
* how does this number compare, for example, to the [number of atoms in the observable universe?](https://en.wikipedia.org/wiki/Observable_universe#Matter_content%E2%80%94number_of_atoms)

In [ ]:
# Ex. 1 your code here...

# To answer these questions, we must first define what it means for a state to be unique.
# For example, it is possible to consider isomorphic shapes to be the same. To be even more extreme,
# variations with the same number of "living" neighbours can all be considered the same macrostate.
# But, if we were to count simple permutations, the results are as follows

unique_states = 2**9             # each cell may either have a value of 0 or 1
unique_rules = 2**unique_states  # each state may lead the next centre to have a value of 0 or 1

message = (f'There are {unique_states} unique states for a 3x3 neighbourhood, '
           f'and to these, {unique_rules:.2e} unique rules can be applied.\n'
           f'Compared to the estimated {10**80:.0e} atoms in the observable universe, '
           f'this number is, in consideration, quite astronomical.')
print(message)

There are 512 unique states for a 3x3 neighbourhood, and to these, 1.34e+154 unique rules can be applied.
Compared to the estimated 1e+80 atoms in the observable universe, this number is, in consideration, quite astronomical.


Using what you learned above, concisely answer each question below (point form)
1. would it be a reasonable goal to conduct an experiment like Wolfram’s for 2D CA’s?
2. propose a numbering scheme for these rules, analogous to Wolfram’s rule numbers
3. how might we go about determining which rule number corresponds to Conway’s GoL?
   (I’m not asking for the number, but rather for a method you could use to find it.)

#### Ex 1. Your Answers

1.  Probably not. The number of different rules that can be devised are simply too many for each to be studied individually. Although, there are probably many rules that share certain properties, and so many can be classified this way.
2.  Not knowing what kinds of symmetries and other simplifications that can be made, the simplest (though highly impractical) numbering scheme would be to use exactly the same process which derived Wolfram's rule numbers.
3.  Like with Wolfram's rule numbers, we could encode each neighbourhood state as a decimal number from 0 to 511. Next, would be the arduous process of filling in which of those states leads to an "alive" centre cell in the next step, or if it proves simpler, which of those states leads to a "death".

## Exercise 2: random_life
Extend the Life2D class defined above to allow the game to be initialized with a random start state.
The liklihood of any given cell being initialized to "alive" is given by $p$.
*Hint*: we did this exact exercise for a 1D array in previous lab

In [12]:
# Ex. 2 experimental setup

class BetterLife2D(Life2D):
    """ Improvements on the basic Life2D model """

    def randomize_state(self, p):
        """ Randomize the state of this GoL, where probability of a "live" cell is p """
        rng = np.random.default_rng()
        (n_rows, n_cols) = self.state.shape  # tuple unpacking
        for i in range(n_rows):  # a loop over the one-dimensional version from the previous lab
            row = np.fromiter((rng.random() < p for i in range(n_cols)), dtype = np.uint8)
            self.state[i] = row


def random_life(n_rows, n_cols=None, p=0.5):
    """ Return a Life2D object of given dimensions with initial randomized state where "live" cells have probability p """
    l = BetterLife2D(n_rows, n_cols)
    l.randomize_state(p)
    return l

brioche = random_life(5)
croissant = random_life(n_rows = 8, n_cols = 12, p = 0.3)

print(brioche.state)
print(croissant.state)


# A few notes to self:
#
# <BetterLife2D> is a derived class of the base class <Life2D>,
# it extends the functionality of <Life2D> by introducing a <random_state()> method
#
# Remember, the original interface was <n_rows>, optional: <n_cols>

[[1 1 0 0 1]
 [0 1 0 0 0]
 [0 1 0 1 0]
 [1 1 1 1 0]
 [1 1 0 0 0]]
[[0 0 0 0 1 1 1 1 1 1 1 0]
 [1 0 1 0 0 1 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 1 0 1 1]
 [0 0 1 0 0 0 1 0 0 0 1 0]
 [1 0 0 0 0 0 1 0 0 0 0 1]
 [0 0 1 0 0 1 1 0 1 0 0 1]
 [1 0 0 0 1 1 0 0 1 0 0 0]]


### Experiment
Run a small experiment to see if you can identify any relationship between $p$, the probability of a live cell in the initial grid, and the time it takes for the GoL pattern to stabilize.

In [14]:
# Ex. 2 write code for your experiment here...

random_start = random_life(8, p = 0.1)
anim = Animation2D(random_start, 30, figsize = (5, 5))
croissant = anim.show()

# could plot a graph (practice matplotlib)

# vector indexing (random notes)

# one = np.array([1,2,3,4,5])
# two = np.array([True, False, False, False, False, True])
# # one[two]

# three = np.array([1,2,3,4,5,6,7])
# filter = np.array([0, 1, 0, 0, 0, 1, 1], dtype = bool)
# three[filter] = 0
# three

# Quick info:
#
# Animate2D class interface: <model>, optional: <frames> (default: 50), optional: <figsize> (default: 8, 8)
# animate(), show()

croissant

## Exercise 3: Load Patterns
People have discovered many interesting ["patterns"](https://conwaylife.com/wiki/Category:Patterns) that exhibit various types of behaviours.
A "pattern" is just a small 2D array.
 1. Develop a function that returns a 2D array containing the patterns defined by a single string; and
 2. Add a `load_pattern(x, y, pattern)` method to your `BetterLife2D` class above to insert patterns into the Life state array
    *Tip*: you can [assign an entire 2D slice of a numpy array](https://www.nickmccullum.com/advanced-python/numpy-indexing-assignment/) in a single assignment statement!   See example in the Sample Usage block above. Try it!

In [19]:
# Here's an example of the glider pattern from above defined in a single string...
glider_pattern = \
    """
    010
    001
    111
    """

glider_pattern

# Ex. 3 your code here...

'\n    010\n    001\n    111\n    '

### Define some patterns
User your nifty new function to define a few common, interesting patterns (e.g., from the book or other sources)

In [ ]:
# Ex. 3 define your patterns here...

### Experiment
Load up a BetterLife2D object with some patterns that interact in some interesting way.
What can we learn from studying interactions like this?   Is anything here applicable to the real world?

In [ ]:
# Ex. 3 code for your experiment here